# form

> this builds up an `.md` with a lot of nastiness to get to an HTML form to propose a new config file

In [ ]:
import re
from pathlib import Path

import importnb
import jinja2

In [ ]:
URL = "https://github.com/deathbeds/jupyak/new/main"

In [ ]:
with importnb.Notebook():
    from jupyak.tasks import load_tasks
    from jupyak.tasks._yak import Repo, Yak

In [ ]:
def load_yak():
    load_tasks()
    return Yak({"pr": {}})

In [ ]:
def make_repo(repo: Repo):
    return []

In [ ]:
REPOS_TMPL = r"""
<h2>repos</h2>
{% for name in yak.repos %}
    <input type="checkbox" name="show-repo-{{ name }}" id="show-repo-{{ name }}"/>
    <label class="show-repo-label" for="show-repo-{{ name }}">{{name}}</label>
{% endfor %}
<table style="position: relative;">
    <thead style="position: sticky; top: 0;">
        <tr>
            <th>repo</th>
            <th>baseline</th>
            <th>merge with</th>
            <th>merge strategy</th>
            <th>merge options</th>
        </tr>
        <tr>
            <td><blockquote>
                GitHub repository to check out and build
            </blockquote></td>
            <td><blockquote>
                starting point GitHub reference
            </blockquote></td>
            <td><blockquote>
                optional space-delimted list of references to merge into the baseline
            </blockquote></td>
            <td><blockquote>
                merge strategy
            </blockquote></td>
            <td><blockquote>
                additional space-delimeted <code>-X</code> options to pass to <code>git merge</code>
            </blockquote></td>
        </tr>
    </thead>
    <tbody>
        {% for name, repo in yak.repos.items() %}
        {% set stem = "repos|" ~ name ~ "|github" %}
        {% set id_stem = "repos-" ~ name ~ "-github" %}
        {% set gh = repo.github %}
        {% set gh_pattern = "(tree/[^s]+|pull/\d+|releases/tag/[^s]+)" %}
        <tr class="repo" id="repo-{{ name }}">
            <th><code>{{ gh.url }}/</code></th>
            <td>
                <input id="{{ id_stem }}-baseline"
                    name="{{ stem }}|baseline"
                    type="text"
                    title="the baseline GitHub URL for {{ name }}"
                    spellcheck="false"
                    placeholder="{{ gh.baseline }}"
                    pattern="^$|^{{ gh_pattern }}"
                />
                <label for="{{ id_stem }}-baseline">
                    must be empty, or one of:<br/>
                    <code>pull/{:number}</code><br/>
                    <code>tree/{:branch}</code><br/>
                    <code>releases/tag/{:tag}</code>
                </label>
            </td>
            <td>
                <input id="{{ id_stem }}-merge_with"
                    name="{{ stem }}|merge_with"
                    title="one or more space-delimited GitHub URLs to merge into the {{ name }} baseline"
                    type="text"
                    spellcheck="false"
                    placeholder="pull/{:number} tree/{:branch} releases/tag/{:tag}"
                    pattern="^$|^{{ gh_pattern }}(\s+{{ gh_pattern}})*"
                />
                <label for="{{ id_stem }}-merge_with">
                    must be empty, or one or more (separated by space) of:<br/>
                    <code>pull/{:number}</code><br/>
                    <code>tree/{:branch}</code><br/>
                    <code>releases/tag/{:tag}</code>
                </label>
            </td>
            <td>
                <select
                    name="{{ stem }}|merge_strategy"
                    title="choose a different git merge strategy"
                >
                    <option value="">ort (default)</option>
                    <option>resolve</option>
                    <option>octopus</option>
                    <option>ours</option>
                    <option>subtree</option>
                </select>
            </td>
            <td>
                <input
                    name="{{ stem }}|merge_options"
                    type="text"
                    title="add space-delimted -X options for the merge strategy"
                />
            </td>
        </tr>
        {% endfor %}
    </tbody>
</table>
"""

In [ ]:
def make_repos_form(yak: Yak):
    return [jinja2.Template(REPOS_TMPL).render(yak=yak)]

In [ ]:
REPO_STYLE_TEMPL = """
{% for name in yak.repos %}
#show-repo-{{ name }}:not(:checked) ~ table tbody #repo-{{ name }} {
    display: none;
}
{% endfor %}
"""

In [ ]:
def make_repos_style(yak: Yak):
    return [jinja2.Template(REPO_STYLE_TEMPL).render(yak=yak)]

In [ ]:
def make_form():
    yak = load_yak()
    chunks = []
    style_chunks = []
    script_chunks = []
    for trait_name, value in yak.trait_values().items():
        maker = globals().get(f"make_{trait_name}_form")
        if maker:
            chunks += maker(yak)
        styler = globals().get(f"make_{trait_name}_style")
        if styler:
            style_chunks += styler(yak)

    return [
        "<style>",
        *style_chunks,
        "</style>",
        """<form id="new">""",
        *chunks,
        "</form>",
        *script_chunks,
    ]

In [ ]:
def write_form(dest: Path, **options):
    dest.parent.mkdir(parents=True, exist_ok=True)
    chunks = [
        "# request a preview site",
        "",
        "> make selections below, then use the sidebar to start a new Pull Request",
        "",
        *make_form(),
    ]
    txt = re.sub(r"^ +", "", "\n".join(chunks), flags=re.M)
    dest.write_text(txt, encoding="utf-8")